<a href="https://colab.research.google.com/github/KimJae-hee/kyakyakyakya/blob/master/%EC%9D%B4%EA%B1%B0%EB%A1%9C%20%EB%B3%B4%EA%B3%A0%20%ED%95%B4%EB%B4%90%EB%9D%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [120]:
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split

# !pip install xgboost
# !pip install category_encoders
# import xgboost as xgb
# import category_encoders

In [159]:
train  = pd.read_csv("/content/drive/My Drive/P-SAT/2020-2/방학 세미나/최종 데이터/cleaned_train.csv")
# test = pd.read_csv("/content/drive/My Drive/P-SAT/2020-2/방학 세미나/최종 데이터/cleaned_test.csv)

In [161]:
train = train.drop(["ssp_id", "device_os", "device_ifa", "campaign_id", "placement_type", "advertisement_id", "age", "gender", "marry", "bid_id"], axis = 1)

In [162]:
train_X = train.drop(['click'], axis = 1)
train_Y = train.loc[:,"click"]

In [163]:
X_use, X_rest, Y_use, Y_rest = train_test_split(train_X, train_Y, test_size=0.8, train_size=0.2, stratify= train_Y)

In [164]:
kfold  = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 1234)

In [ ]:
param = {}
param['booster'] = 'gbtree'
param['objective'] = 'binary:logistic'
param["eval_metric"] = "error"
param['eta'] = 0.3
param['gamma'] = 0
param['max_depth'] = 6
param['min_child_weight']=1
param['max_delta_step'] = 0
param['subsample']= 1
param['colsample_bytree']=1
param['silent'] = 1
param['seed'] = 0
param['base_score'] = 0.5

In [149]:
result_list = []
for train_idx, test_idx in kfold.split(X_use, Y_use):
  train_fold_X, test_fold_X = X_use.iloc[train_idx,:], X_use.iloc[test_idx,:]
  train_fold_Y, test_fold_Y = Y_use.iloc[train_idx], Y_use.iloc[test_idx]
  print(f"폴드 나누기 성공")
  train_x, val_x, train_y, val_y = train_test_split(train_fold_X, train_fold_Y, test_size = 0.2, train_size = 0.8, stratify = train_fold_Y)
  catboost_encoder = category_encoders.CatBoostEncoder()
  train_cbe = catboost_encoder.fit_transform(train_x, train_y)
  val_cbe = catboost_encoder.transform(val_x)
  print(f"인코딩 성공")
  weight_ratio = float(len(train_y[train_y == 0]))/float(len(train_y[train_y == 1]))
  w_array = np.array([1]*train_y.shape[0])
  w_array[train_y==1] = weight_ratio
  w_array[train_y==0] = 1- weight_ratio

  eval_set = [(val_cbe, val_y)]
  print(f"모델 준비 성공")

  model = xgb.XGBClassifier(random_state = 1234, n_estimators = 500)
  model.fit(train_cbe, train_y, eval_metric = "logloss", eval_set = eval_set, early_stopping_rounds = 50)
  test_cbe = catboost_encoder.transform(test_fold_X)
  pred_fold = model.predict_proba(test_cbe)
  print(f"모델 예측 성공")
  result_tmp = log_loss(test_fold_Y, pred_fold)
  result_list.append(result_tmp)
  print(f"결과 저장 성공")
print(np.mean(result_list))

폴드 나누기 성공
인코딩 성공
모델 준비 성공
[0]	validation_0-logloss:0.6213
Will train until validation_0-logloss hasn't improved in 50 rounds.
[1]	validation_0-logloss:0.562566
[2]	validation_0-logloss:0.513666
[3]	validation_0-logloss:0.472681
[4]	validation_0-logloss:0.437929
[5]	validation_0-logloss:0.408178
[6]	validation_0-logloss:0.382697
[7]	validation_0-logloss:0.360698
[8]	validation_0-logloss:0.34167
[9]	validation_0-logloss:0.325112
[10]	validation_0-logloss:0.310742
[11]	validation_0-logloss:0.29821
[12]	validation_0-logloss:0.287254
[13]	validation_0-logloss:0.277683
[14]	validation_0-logloss:0.269414
[15]	validation_0-logloss:0.262075
[16]	validation_0-logloss:0.255665
[17]	validation_0-logloss:0.250001
[18]	validation_0-logloss:0.245098
[19]	validation_0-logloss:0.240762
[20]	validation_0-logloss:0.236963
[21]	validation_0-logloss:0.233643
[22]	validation_0-logloss:0.230664
[23]	validation_0-logloss:0.228132
[24]	validation_0-logloss:0.225975
[25]	validation_0-logloss:0.224041
[26]	valid

In [ ]:
train_fold_X, test_fold_X, train_fold_Y, test_fold_Y = train_test_split(train_X, train_Y, test_size=0.2, train_size=0.8, stratify= train_Y)
print(f"폴드 나누기 성공")

train_x, val_x, train_y, val_y = train_test_split(train_fold_X, train_fold_Y, test_size = 0.2, train_size = 0.8, stratify = train_fold_Y)
catboost_encoder = category_encoders.CatBoostEncoder()
train_cbe = catboost_encoder.fit_transform(train_x, train_y)
val_cbe = catboost_encoder.transform(val_x)
print(f"인코딩 성공")

weight_ratio = float(len(train_y[train_y == 0]))/float(len(train_y[train_y == 1]))
w_array = np.array([1]*train_y.shape[0])
w_array[train_y==1] = weight_ratio
w_array[train_y==0] = 1- weight_ratio
eval_set = [(val_cbe, val_y)]
print(f"모델 준비 성공")

model = xgb.XGBClassifier(random_state = 1234, n_estimators = 500)
model.fit(train_cbe, train_y, eval_metric = "logloss", eval_set = eval_set, early_stopping_rounds = 50)
test_cbe = catboost_encoder.transform(test_fold_X)
pred_fold = model.predict_proba(test_cbe)
print(f"모델 예측 성공")

result_tmp = log_loss(test_fold_Y, pred_fold)
print(f"결과 저장 성공")

폴드 나누기 성공
인코딩 성공
모델 준비 성공
[0]	validation_0-logloss:0.621582
Will train until validation_0-logloss hasn't improved in 50 rounds.
[1]	validation_0-logloss:0.562463
[2]	validation_0-logloss:0.513944
[3]	validation_0-logloss:0.472862
[4]	validation_0-logloss:0.438246
[5]	validation_0-logloss:0.408418
[6]	validation_0-logloss:0.38325
[7]	validation_0-logloss:0.361139
[8]	validation_0-logloss:0.342329
[9]	validation_0-logloss:0.325739
[10]	validation_0-logloss:0.311328
[11]	validation_0-logloss:0.298827
[12]	validation_0-logloss:0.287849
[13]	validation_0-logloss:0.278356
[14]	validation_0-logloss:0.270091
[15]	validation_0-logloss:0.2628
[16]	validation_0-logloss:0.256363
[17]	validation_0-logloss:0.250723
[18]	validation_0-logloss:0.245921
[19]	validation_0-logloss:0.241674
[20]	validation_0-logloss:0.237861
[21]	validation_0-logloss:0.234652
[22]	validation_0-logloss:0.231703
[23]	validation_0-logloss:0.2293
[24]	validation_0-logloss:0.227072
[25]	validation_0-logloss:0.225148
[26]	valida

In [156]:
print(f"best iteration is{model.best_iteration}")
pd.DataFrame({'importance': model.feature_importances_,
              'features': train_fold_X.columns})
["ssp_id", "device_os", "device_ifa", "campaign_id", "placement_type", "advertisement_id", "age", "gender", "marry", "bid_id"]

best iteration is69


,importance,features
0,0.016214,ssp_id
1,0.013589,campaign_id
2,0.019940,adset_id
3,0.016357,placement_type
4,0.356327,media_id
5,0.084734,media_name
6,0.096035,media_bundle
7,0.009346,publisher_id
8,0.246707,publisher_name
9,0.000000,device_ifa
